In [ ]:
!pip install -U ultralytics wandb

In [ ]:
import random
import shutil
import torch
import yaml
import os

import numpy as np

from pathlib import Path

from ultralytics import YOLO

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
shutil.copy2('/content/drive/MyDrive/ITaS/data/best.pt', 'best.pt')

In [ ]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.login(key='1de7addec37341330a8aef9bfc7be382cc9c2824')

In [ ]:
def create_yaml(output_yaml_path, train_image_dir, val_image_dir, test_image_dir=' ', nc=10):

    names = ['lungs', 'trachea', 'bronchitis', 'pneumonia/bronchopneumonia', 'pulmonary edema',
             'hydrothorax', 'pneumothorax', 'tracheal collapse', 'neoplasm', 'atelectasis']

    yaml_data = {
        'names': names,
        'nc': nc,
        'train': train_image_dir,
        'val': val_image_dir,
        'test': test_image_dir
    }

    with open(output_yaml_path, 'w') as j:
        yaml.dump(yaml_data, j, default_flow_style=False)

In [ ]:
def calculation_margin_sampling_dataset(
    confidence_threshold: float = 0.01
):
    path_to_dataset = '/content/drive/MyDrive/ITaS/data/full_train'

    # train_labels_paths не создаем, так как пройдя через алгоритм Least Confidence
    # мы получим урезанный список картинок, которму train_labels_paths не будет соответствовать.
    # Легче потом просто использовать str.replace(), так как наименование файлов одинаковое.
    train_images_paths = []

    val_images_paths = []
    val_labels_paths = []

    # Проходимся по директории полного датасета
    for root, _, files in os.walk(path_to_dataset):
        for file in files:
            # train часть
            if r'/train/images' in root:
                train_images_paths.append(os.path.join(root, file))
            # val часть
            elif r'/val/images' in root:
                val_images_paths.append(os.path.join(root, file))
            elif r'/val/labels' in root:
                val_labels_paths.append(os.path.join(root, file))

    # Алгоритм Least Confidence
    model = YOLO('best.pt')

    selected_train_images_samples = []
    for i, image_path in enumerate(train_images_paths):
        print(f'{i}/{len(train_images_paths)} ->> {image_path}')
        results = model.predict(image_path, conf=confidence_threshold)
        result = results[0]

        # Извлекаем номера классов предсказания
        classes = result.boxes.cls

        # pneumonia/bronchopneumonia
        results_pneumonia = result[[i for i, cls in enumerate(classes) if cls in [3.]]]

        if len(results_pneumonia) == 0:
            continue

        result_pneumonia_min = results_pneumonia[[i for i, confidence in enumerate(results_pneumonia.boxes.conf) if confidence == min(results_pneumonia.boxes.conf)]]
        result_pneumonia_max = results_pneumonia[[i for i, confidence in enumerate(results_pneumonia.boxes.conf) if confidence == max(results_pneumonia.boxes.conf)]]

        # pulmonary edema
        results_edema = result[[i for i, cls in enumerate(classes) if cls in [4.]]]

        if len(results_edema) == 0:
            continue

        result_edema_min = results_edema[[i for i, confidence in enumerate(results_edema.boxes.conf) if confidence == min(results_edema.boxes.conf)]]
        result_edema_max = results_edema[[i for i, confidence in enumerate(results_edema.boxes.conf) if confidence == max(results_edema.boxes.conf)]]

        # Выискиываем минимальное расстояние между классами
        max_pneumonia_min_edema = abs(result_pneumonia_max.boxes.conf - result_edema_min.boxes.conf)
        max_edema_min_pneumonia = abs(result_edema_max.boxes.conf - result_pneumonia_min.boxes.conf)

        if max_pneumonia_min_edema > max_edema_min_pneumonia:
            selected_train_images_samples.append({'image_path': image_path, 'min_pneumonia_edema_margin': max_edema_min_pneumonia})
        else:
            selected_train_images_samples.append({'image_path': image_path, 'min_pneumonia_edema_margin': max_pneumonia_min_edema})

    # Сортируем по приоритету
    selected_train_images_samples.sort(key=lambda x: x['min_pneumonia_edema_margin'])

    # Вычленяем только пути у сортированного словаря
    selected_train_images_samples = [image['image_path'] for image in selected_train_images_samples]
    selected_train_labels_samples = [path.replace('images', 'labels').replace('.jpg', '.txt') for path in selected_train_images_samples]

    return (selected_train_images_samples, selected_train_labels_samples, val_images_paths, val_labels_paths)

In [ ]:
margin_sampling_full_dataset = calculation_margin_sampling_dataset()

In [ ]:
def create_margin_sampling_dataset(
    train_images_path: list,
    train_labels_path: list,
    val_images_path: list,
    val_labels_path: list,
    experiment_name: str,
    percentage_from_top: int = None,
    number_from_top: int = None,
    alg_name: str = 'margin_sampling'
):
    if percentage_from_top:
        train_images_path = train_images_path[:int(len(train_images_path) * (percentage_from_top / 100))]
        train_labels_path = train_labels_path[:int(len(train_labels_path) * (percentage_from_top / 100))]
    elif number_from_top:
        train_images_path = train_images_path[:number_from_top]
        train_labels_path = train_labels_path[:number_from_top]

    train_images_dir = os.path.join('data', alg_name, experiment_name, 'train', 'images')
    val_images_dir = os.path.join('data', alg_name, experiment_name, 'val', 'images')
    train_labels_dir = os.path.join('data', alg_name, experiment_name, 'train', 'labels')
    val_labels_dir = os.path.join('data', alg_name, experiment_name, 'val', 'labels')

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    for train_image_path, train_label_path in zip(train_images_path, train_labels_path):
        shutil.copy2(train_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'images',
                          train_image_path[train_image_path.find('images') + 7:]))

        shutil.copy2(train_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'labels',
                          train_label_path[train_label_path.find('labels') + 7:]))

    for val_image_path, val_label_path in zip(val_images_path, val_labels_path):

        shutil.copy2(val_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'images',
                          val_image_path[val_image_path.find('images') + 7:]))

        shutil.copy2(val_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'labels',
                          val_label_path[val_label_path.find('labels') + 7:]))

    yaml_path = os.path.join('data', alg_name, experiment_name, 'data.yaml')
    train_path = os.path.join('train', 'images')
    val_path = os.path.join('val', 'images')
    create_yaml(yaml_path, train_path, val_path)


In [ ]:
create_margin_sampling_dataset(
    train_images_path=margin_sampling_full_dataset[0],     # train_images
    train_labels_path=margin_sampling_full_dataset[1],     # train_labels
    val_images_path=margin_sampling_full_dataset[2],       # val_images
    val_labels_path=margin_sampling_full_dataset[3],       # val_samples
    number_from_top=1,
    experiment_name='1_train'
)

In [ ]:
create_margin_sampling_dataset(
    train_images_path=margin_sampling_full_dataset[0],     # train_images
    train_labels_path=margin_sampling_full_dataset[1],     # train_labels
    val_images_path=margin_sampling_full_dataset[2],       # val_images
    val_labels_path=margin_sampling_full_dataset[3],       # val_samples
    number_from_top=10,
    experiment_name='10_train'
)

In [ ]:
create_margin_sampling_dataset(
    train_images_path=margin_sampling_full_dataset[0],     # train_images
    train_labels_path=margin_sampling_full_dataset[1],     # train_labels
    val_images_path=margin_sampling_full_dataset[2],       # val_images
    val_labels_path=margin_sampling_full_dataset[3],       # val_samples
    number_from_top=20,
    experiment_name='20_train'
)

In [ ]:
def train_yolo_model(main_experiment: str, name: str, seed: int):
    wandb.init(project='ITaS', job_type='training')

    model = YOLO("yolov8m-seg.pt")

    results = model.train(
        data = os.path.join('/content/drive/MyDrive/ITaS' ,'data', main_experiment, name, 'data.yaml'),
        project = 'ITaS',
        name = name,
        epochs = 25,
        patience = 0,
        batch = 5,
        imgsz = 640,
        seed=seed
    )
    wandb.finish()

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='margin_sampling', name=f'1_train', seed=(i + 1) * 7)

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='margin_sampling', name=f'10_train', seed=(i + 1) * 8)

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='margin_sampling', name=f'20_train', seed=(i + 1) * 9)